Parallel Map on Files
------------------------

For each of a set of filenames, we parse JSON data contents, load that data into a Pandas DataFrame, and then output the result to another file with a nicer format, HDF5.

We find that parsing JSON is slow and so we parallelize the process using the [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) module to do this work in multiple processes.

### Objectives

*  Profile code to find bottleneck
*  Use `concurrent.futures` to `map` a function across many inputs in parallel


### Requirements

*  Pandas
*  concurrent.futures (standard in Python 3, `pip install futures` in Python 2)
*  snakeviz (for profile visualization, `pip install snakeviz`)


    pip install snakeviz
    pip install futures
    
### Extra Exercise

Try out alternative binary formats.  Perhaps try [feather](https://github.com/wesm/feather).

## Sequential Execution

In [2]:
%load_ext snakeviz

In [3]:
from glob import glob
import json
import pandas as pd
import os

In [ ]:
filenames = sorted(glob(os.path.join('..', 'data', 'json', '*.json')))  # ../data/json/*.json
filenames[:5]

In [4]:
%%time

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    
    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')

CPU times: user 6.25 s, sys: 208 ms, total: 6.46 s
Wall time: 13.6 s


Parallel Execution
--------------------

We can process each file independently and in parallel.  To accomplish this we'll transform the body of our for loop into a function and then use the [concurrent.futures.ProcessPoolExecutor](https://docs.python.org/3/library/concurrent.futures.html#executor-objects) to apply that function across all of the filenames in parallel using multiple processes.

### Before

Whenever we have code like the following:

```python
results = []
for x in L:
    results.append(f(x))
```

or the following:

```python
results = [f(x) for x in L]
```

or the following:

```python
results = list(map(f, x))
```

### After

We can instead write it as the following:

```python
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

results = list(e.map(f, L))
```

### Example

In [22]:
%%time

### Sequential code

from time import sleep

results = []

for i in range(8):
    sleep(1)
    results.append(i + 1)

In [23]:
%%time

### Parallel code

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

def slowinc(x):
    sleep(1)
    return x + 1

results = list(e.map(slowinc, range(8)))

CPU times: user 24 ms, sys: 20 ms, total: 44 ms
Wall time: 2.05 s


### Exercise:  Convert JSON data to HDF5 in parallel using `concurrent.futures.Executor.map`

In [ ]:
%%time

### Sequential code

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    
    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')

In [ ]:
%%time

### Parallel code

# TODO

In [34]:
%load solutions/map-1.py

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Parallelism isn't everything
--------------------------------

We get a moderate increase in performance when using multiple processes.  However parallelism isn't the only way to accelerate this computation.  Recall that the bulk of the cost comes from the `json.loads` function.  A quick internet search on "fast json parsing in python" yields the [ujson](https://pypi.python.org/pypi/ujson) library as the top hit.

Knowing about and importing the optimized `ujson` library is just as effective as multi-core execution.

In [8]:
import ujson as json

In [9]:
%%time
filenames = sorted(glob(os.path.join('..', 'data', 'json', '*.json')))

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    
    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')

CPU times: user 2.32 s, sys: 92 ms, total: 2.42 s
Wall time: 2.42 s


History: multiprocessing.Pool
--------------------------------

Perviously people have done multi-processing computations with the `multiprocessing.Pool` object, which behaves more or less identically.

However, today most library designers are coordinating around the `concurrent.futures` interface, so it's wise to move over.

In [11]:
%%time 

from multiprocessing import Pool
p = Pool()

list(p.map(load_parse_store, filenames))

CPU times: user 20 ms, sys: 24 ms, total: 44 ms
Wall time: 1.39 s


Conclusion
-----------

*  Used `snakeviz` to profile code
*  Used `concurrent.futures.ProcessPoolExecutor` for simple parallelism across many files
    *  Gained some speed boost (but not as much as expected)
    *  Lost ability to diagnose performance within parallel code
*  Saw that other options than parallelism exist to speed up code, including the `ujson` library.